# Data Preprocessing

## Initialization

In [1]:
import numpy as np
import pandas as pd
import dill
from tqdm import tqdm_notebook

## Rolling Average Transformation

We want to present the MAXWEL model with stationary timeseries data, in order to do so, we will use a 7 day rolling average.  This approach has the advantage that the rolling average (which we will model separately from the day-to-day data) changes fairly slowly, and serves as a strong proxy for seasonal effects.  

### Demand Data

In [14]:
def build_scaled_demand_data():
    # Load in the relevant data
    station_metadata_df=pd.read_csv("Data/Metadata/station_meta_data.csv")
    names=station_metadata_df['station_tag'].unique()
    demand_data=pd.read_csv('Data/Demand_Data/Raw_Data/EIA_demand_data.csv',converters={'Unnamed: 0':pd.to_datetime})
    demand_data=demand_data.rename(index=str, columns={"Unnamed: 0":'time'}).set_index('time')
    
    # Drop nonsense values and interpolate
    demand_data[demand_data<0]=np.NAN
    demand_data=demand_data.interpolate()
    
    #Compute the 7 day rolling average for each station
    for name in names:
        demand_data[f'{name}_rolling_mean']=demand_data[f'{name}_D'].rolling(7*24).mean()
        demand_data[f'{name}_rolling_std']=demand_data[f'{name}_D'].rolling(7*24).std()
    
    # Drop the NaN values
    rolling_demand_data=demand_data[7*24-1:]

    # Build the stationary dataframe
    demand_out=pd.DataFrame()
    for name in tqdm_notebook(names):
        demand_out[f'{name}_scaled_demand']=(rolling_demand_data[f'{name}_D']-rolling_demand_data[f'{name}_rolling_mean'])/rolling_demand_data[f'{name}_rolling_std']
        demand_out[f'{name}_rolling_mean']=rolling_demand_data[f'{name}_rolling_mean']
        demand_out[f'{name}_rolling_std']=rolling_demand_data[f'{name}_rolling_std']
        demand_out[f'{name}_scaled_DA']=(rolling_demand_data[f'{name}_DF']-rolling_demand_data[f'{name}_rolling_mean'])/rolling_demand_data[f'{name}_rolling_std']
    
    # Output the csv
    demand_out.to_csv('Data/Demand_Data/Processed_Data/processed_demand_with_DA.csv')

In [15]:
build_scaled_demand_data()

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

### Weather Data

We will perform the same type of rolling average to the weather data but in this case we don't need to save the rolling average information, so the process is more straightforward.

In [6]:
def build_scaled_weather_data():
    """This function processes and saves all of the collated weather data"""
    # Load in the metadata
    station_metadata_df=pd.read_csv("Data/Metadata/station_meta_data.csv")
    city_names=list(map(lambda x:x.split(',')[0],station_metadata_df['city'].unique()))
    
    # Perform RA transformation on each city
    for city in tqdm_notebook(city_names):
        df=pd.read_csv(f'Data/Weather_Data/Collated_Data/{city}.csv')
        out=process_weather(df)
        out=out.interpolate().fillna(0)
        out.to_csv(f'Data/Weather_Data/Processed_Data/{city}.csv')
        
def process_weather(df_in):
    """This function processes a single weather dataframe"""
    
    df_out=pd.DataFrame()
    
    # Select and process the desired columns
    df_out['time']=df_in['time']
    df_out['temperature']=shmear(df_in.temperature)
    df_out['windspeed']=shmear(df_in.windSpeed)
    df_out['pressure']=shmear(df_in.pressure)
    
    # These three columns have values in [0,1], hence we don't rescale them
    df_out['precip']=df_in.precipProbability.fillna(0)
    df_out['severity']=df_in.precipIntensity.fillna(0)
    df_out['humidity']=df_in.humidity
    
    # Drop NaNs and interpolate
    df_out=df_out[9*24:].interpolate()
    df_out=df_out.set_index('time')
    
    return df_out

def shmear(col):
    """This function performs the 7 day rolling average"""
    return (col-col.rolling(7*24).mean())/(col.rolling(7*24).std())

In [7]:
build_scaled_weather_data()

HBox(children=(IntProgress(value=0, max=44), HTML(value='')))

## MAXWEL Training Data

MAXWEL takes a 24 window of demand data and weather data, and makes a 4 hour prediction. The demand data that MAXWEL recives contains the station it is training on, along with all of the interchange stations for the trainging station; i.e. the ERCO station model sees the ERCO data with all of the stations connected to the ERCO station.

The following function packages this data in a csv file.

In [16]:
def build_MAXWEL_train_data():
    """This function assembles the final training datasets."""
    
    # Load in the relevant data
    df=pd.read_csv('Data/Demand_Data/Processed_Data/processed_demand_with_DA.csv',
                   converters={'time':pd.to_datetime}).set_index('time')
    station_metadata_df=pd.read_csv("Data/Metadata/station_meta_data.csv" ).set_index('station_tag')
    names=station_metadata_df.index

    
    for name in tqdm_notebook(names):
        # Pull the appropriate weather data
        weather_city=(station_metadata_df.loc[name].city).split(',')[0].strip()
        weather_data=pd.read_csv(f'Data/Weather_Data/Processed_Data/{weather_city}.csv',
                                converters={'time':pd.to_datetime}).set_index('time')
        # Pull the appropriate interchange data
        inter=list(filter(lambda x: x in names ,station_metadata_df.loc[name][[f'interchange_{i}' for i in range(1,16)]].values))
        inter.append(name)
        
        # Join the demand and weather data, save the result
        demand_data=df[[f'{guy}_scaled_demand' for guy in inter]]
        join_df=pd.merge(demand_data,weather_data,how='inner',on='time')
        join_df.to_csv(f'Data/MAXWEL_Data/{name}.csv')

In [17]:
build_MAXWEL_train_data()

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))